In [1]:

import tensorflow as tf
from tensorflow.keras import layers, models
import os

C:\conda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Create image data from folder using image_dataset_from_directory function form keras. Validation split is set at 0.2 
#which means 80% of the data is used for training and 20% for validation. Images are converted into grayscale since its
#black and white drawings of Mongolian traditional character
image_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory= r"C:\Users\Galaxy Book Ion\Desktop\media",
    labels='inferred',
    label_mode='int',
    batch_size=2,
    image_size=(300, 300),
    color_mode = "grayscale",
    shuffle=True,
    validation_split=0.2,
    seed=0,
    subset="both",
)

Found 44 files belonging to 2 classes.
Using 36 files for training.
Using 8 files for validation.


In [3]:
#Split the training and validation data from image_data tuple
training_data = image_data[0]

val_data = image_data[1]

In [7]:
#Creating simple machine learning model to test if our gathered dataset can be trained on simple baseline model
model = models.Sequential([
    #First layer of convolution
    layers.Conv2D(4, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    #Second layer of convolution
    layers.Conv2D(8, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    #Flatten the data into single dimension
    layers.Flatten(),
    #Fully connected layers with single node since we only have two classes for the prototype (Letter A and O)
    layers.Dense(64, activation='relu'),
    layers.Dense(18, activation='softmax')    
])

In [8]:
#Used "adam" optimizer since we are testing baseline simple model with binary crossentropy for loss function since we binary 
#classifcation(Letter A or O)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
#Used small number of epochs for prototype to show its possible to train the model on our gathered dataset
history = model.fit(
    training_data,
    validation_data=val_data,
    epochs=10
)

Epoch 1/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.4825 - loss: 391.3728 - val_accuracy: 0.3750 - val_loss: 285.5713
Epoch 2/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step - accuracy: 0.6239 - loss: 145.8077 - val_accuracy: 0.6250 - val_loss: 38.3810
Epoch 3/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - accuracy: 0.7942 - loss: 23.8944 - val_accuracy: 0.8750 - val_loss: 5.8377
Epoch 4/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.8558 - loss: 15.8633 - val_accuracy: 0.7500 - val_loss: 23.2434
Epoch 5/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.9035 - loss: 7.4330 - val_accuracy: 0.6250 - val_loss: 30.6800
Epoch 6/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8175 - loss: 3.8706 - val_accuracy: 1.0000 - val_loss: 4.9572e-14
Epoch 7/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 0.8750 - val_loss: 4.2335
Epoch 8/10
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 1.0000 - loss: 1.8069e-13 - v